source : https://github.com/ahmed-BH/TextureBasedImageRetriever/blob/master/cbir.py

In [ ]:
import numpy as np
import cv2
import os, glob, math
from sklearn.metrics.pairwise import euclidean_distances

dataset_dir = "poster"
descriptors_dir = "feature"

if not os.path.exists(descriptors_dir):
    os.makedirs(descriptors_dir)


generate descriptor file

In [ ]:
def get_descriptor(image_file):
    kernel        = cv2.getGaborKernel((21, 21), 8.0, np.pi/4, 10.0, 0.5, 0, ktype=cv2.CV_32F)
    kernel       /= math.sqrt((kernel * kernel).sum())
    ori_img       = cv2.imread(image_file)
    image         = cv2.cvtColor(ori_img, cv2.COLOR_BGR2GRAY)
    image         = cv2.resize(image, (500, 750), interpolation = cv2.INTER_LINEAR)
    filtered_img  = cv2.filter2D(image, cv2.CV_8UC3, kernel)
    heigth, width = kernel.shape
    
    # convert matrix to vector 
    texturedesc = cv2.resize(filtered_img, (3*width, 3*heigth), interpolation=cv2.INTER_CUBIC) / 255

    hsv = cv2.cvtColor(ori_img,cv2.COLOR_BGR2HSV)
    hue = cv2.calcHist([hsv], [0], None, [180], [0, 180])
    sat = cv2.calcHist([hsv], [1], None, [256], [0, 256])
    colordesc = np.hstack(np.concatenate((hue, sat), axis = None))
    colordesc /= colordesc.max()
    colordesc *= 10

    descriptor = np.hstack(np.concatenate((texturedesc, colordesc), axis = None))
    return descriptor

all_descriptors = []
for entry in glob.glob(os.path.join(dataset_dir,"*.jpg")):
    print(entry)
    descriptor = get_descriptor(entry)
    descriptor_file = os.path.join(descriptors_dir,"{}.npy".format(os.path.basename(entry)))
    np.save(descriptor_file, descriptor, allow_pickle=False)
    # print(descriptor.shape)

testing

In [ ]:
test_descriptor = get_descriptor("poster\\zDduhCHasKQ9YOTvlOreHem7Wbi.jpg")
best_fit_images = []

for descriptorfile in glob.glob(os.path.join(descriptors_dir,"*.npy")):
    descriptor = np.load(descriptorfile)
    distance = abs(euclidean_distances(descriptor.reshape(1, -1), test_descriptor.reshape(1, -1)))
    entry = {"image_path": descriptorfile.replace(".npy","").replace(descriptors_dir,""),
             "distance"  : distance
            }
    best_fit_images.append(entry)
    best_fit_images.sort(key= lambda x:x["distance"], reverse=False)
    best_fit_images = best_fit_images[:10]
# print(best_fit_images)
fit_file = [row["image_path"] for row in best_fit_images]
print(fit_file)

texture feature extraction

In [ ]:
kernel        = cv2.getGaborKernel((21, 21), 8.0, np.pi/4, 10.0, 0.5, 0, ktype=cv2.CV_32F)
kernel       /= math.sqrt((kernel * kernel).sum())
ori_img       = cv2.imread("poster\\zDduhCHasKQ9YOTvlOreHem7Wbi.jpg")
image         = cv2.cvtColor(ori_img, cv2.COLOR_BGR2GRAY)
image         = cv2.resize(image, (500, 750), interpolation = cv2.INTER_LINEAR)
filtered_img  = cv2.filter2D(image, cv2.CV_8UC3, kernel)
heigth, width = kernel.shape

# convert matrix to vector 
texturedesc = cv2.resize(filtered_img, (3*width, 3*heigth), interpolation=cv2.INTER_CUBIC) / 255
print(np.hstack(texturedesc))
r = np.ptp(texturedesc)
print(r)
print(texturedesc.max())
print(texturedesc.min())

color feature extraction

In [ ]:
img = cv2.imread("poster\\zDduhCHasKQ9YOTvlOreHem7Wbi.jpg")
hsv = cv2.cvtColor(ori_img,cv2.COLOR_BGR2HSV)
hue = cv2.calcHist([hsv], [0], None, [180], [0, 180])
sat = cv2.calcHist([hsv], [1], None, [256], [0, 256])
colordesc = np.hstack(np.concatenate((hue, sat), axis = None))
colordesc /= colordesc.max()
colordesc *= 10
print(colordesc)
r = np.ptp(colordesc)
print(r)
print(colordesc.max())
print(colordesc.min())